In [17]:
import datetime
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, log_loss

In [18]:
full_df = pd.read_pickle("../ncaa_data/full_df.p")

# Filter Down to Rows Where Each Team Has Actually Played a Previous Game So We Have Data

In [3]:
populated_df = full_df[(full_df["FGM"] > 0) & (full_df["FGM2"] > 0)]

In [4]:
exclude = ["Team1Score","Team2Score", "date","DayZero","Score","DayNum","Season","WScore","LScore","Team1","Team2","DayZero2","Score2","DayNum2","Season2","date2","index","index2"]
feature_columns = populated_df.columns
feature_columns = [item for item in feature_columns if item not in exclude]

# For a split with only the last tournament as test data 

In [5]:

train_df = populated_df[(populated_df["DayNum"] < 134) | (populated_df["Season"] != 2018)]
test_df = populated_df[(populated_df["DayNum"] >= 134) & (populated_df["Season"] == 2018)]

x_train = train_df[feature_columns]
x_test = test_df[feature_columns]
y = populated_df["Team1Score"] - populated_df["Team2Score"]
y_train = train_df["Team1Score"] - train_df["Team2Score"]
y_test = test_df["Team1Score"] - test_df["Team2Score"]
y_train[y_train < 0] = 0
y_train[y_train > 0] = 1
y_test[y_test < 0] = 0
y_test[y_test > 0] = 1
y[y < 0] =0
y[y> 0 ] =1

In [6]:
x_train.tail(10)

,Ast,Blk,DR,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,AWS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BNT,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,COX,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INC,INP,ISR,JCI,JEN,JJK,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LAB,LAW,LEF,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGS,MGY,MIC,MKV,MMG,MOR,MPI,MSX,MUZ,MvG,NET,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PIR,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STM,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WLS,WMR,WMV,WOB,WOL,WTE,YAG,ZAM,Ast2,Blk2,DR2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,AWS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BNT2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,CNG2,COL2,COX2,CPA2,CPR2,CRO2,CRW2,CTL2,D1A2,DAV2,DC2,DCI2,DDB2,DES2,DII2,DOK2,DOL2,DUN2,DWH2,EBB2,EBP2,ECK2,ENT2,ERD2,ESR2,FAS2,FMG2,FSH2,GC2,GRN2,GRS2,HAS2,HAT2,HER2,HKB2,HKS2,HOL2,HRN2,IMS2,INC2,INP2,ISR2,JCI2,JEN2,JJK2,JNG2,JON2,JRT2,KBM2,KEL2,KLK2,KMV2,KOS2,KPI2,KPK2,KRA2,LAB2,LAW2,LEF2,LMC2,LOG2,LYD2,LYN2,MAS2,MB2,MCL2,MGS2,MGY2,MIC2,MKV2,MMG2,MOR2,MPI2,MSX2,MUZ2,MvG2,NET2,NOL2,NOR2,OCT2,OMY2,PEQ2,PGH2,PH2,PIG2,PIR2,PKL2,PMC2,POM2,PPR2,PRR2,PTS2,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STM2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WLS2,WMR2,WMV2,WOB2,WOL2,WTE2,YAG2,ZAM2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,AWSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BNTdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,COXdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INCdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JJKdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LABdiff,LAWdiff,LEFdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGSdiff,MGYdiff,MICdiff,MKVdiff,MMGdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NETdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PIRdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STMdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WLSdiff,WMRdiff,WMVdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
88524,13.80,5.65,29.65,61.15,24.20,27.05,7.20,0.304496,0.454823,19.00,13.25,0.702222,11.75,15.15,4.75,13.60,0.429585,1.0,1.0,274.0,275.0,1.0,1.0,242.0,1.00,1.00,284.0,313.0,1.0,226.0,327.0,115.0,1.0,278.0,279.0,228.0,192.0,291.0,1.0,1.0,198.0,330.0,1.0,1.00,316.0,308.0,285.0,296.0,278.0,292.0,66.0,1.00,262.0,1.0,269.0,1.0,1.00,1.0,1.0,1.00,174.0,121.0,2.00,238.0,315.0,265.0,257.0,1.0,1.00,1.0,248.0,259.0,187.0,2.00,209.0,320.0,270.0,283.0,266.0,255.0,273.0,1.00,283.0,268.0,170.0,200.0,1.00,1.00,253.0,275.0,275.0,282.0,233.0,282.0,165.0,1.0,1.00,1.0,332.0,1.00,1.00,2.0,1.00,222.0,202.0,1.0,177.0,250.0,1.00,254.0,237.0,232.0,2.0,1.0,255.0,276.0,295.0,292.0,1.00,282.0,264.0,291.0,191.0,253.0,1.00,271.0,2.0

# Split It Up and Train a Model On All Games

In [ ]:
models = [
            (GradientBoostingClassifier(), {"n_estimators":[10,1000, 10000],"max_depth":[3]}),
            (LogisticRegression(), {"C":[.01,1,100,1000]}),
            (SGDClassifier(), {"alpha":[0.1, 10, 100], "loss":['log'], "penalty":['elasticnet','l1','l2']}),
            (RandomForestClassifier(), {"n_estimators":[10,1000]})
         ]

for model_tuple in models:
    gs_cv = GridSearchCV(model_tuple[0], model_tuple[1], cv=5, scoring="neg_log_loss")
    gs_cv.fit(populated_df[feature_columns].fillna(0), y)
    print("Best parameters set found on development set:")
    print()
    print(gs_cv.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    print(gs_cv.best_score_)


In [9]:
# x_train, x_test, y_train, y_test = train_test_split(populated_df[feature_columns], y, test_size=0.1)

In [10]:
model = LogisticRegression(C=100)
model.fit(x_train.fillna(0), y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
predicted = model.predict(x_test.fillna(0))
probs = model.predict_proba(x_test.fillna(0))

In [12]:
predictions = pd.concat([pd.Series([prob[0] for prob in probs]), test_df['Team1'].reset_index(drop=True), test_df['Team2'].reset_index(drop=True)], axis=1, ignore_index=True)

In [13]:
predictions.columns = ['Predicted', 'team1' , 'team2']

In [14]:
team_data = pd.read_csv("../ncaa_data/womens-machine-learning-competition-2019/WTeams.csv")
predictions = predictions.join(team_data.set_index('TeamID'), on='team2', how='left').join(team_data.set_index('TeamID'), lsuffix="team1", on='team1', how='left')

In [15]:
predictions[['TeamNameteam1', 'TeamName', 'Predicted']].head()

,TeamNameteam1,TeamName,Predicted
0,NaN,NaN,0.620033
1,NaN,NaN,0.658784
2,NaN,NaN,0.623748
3,NaN,NaN,0.572662
4,NaN,NaN,0.599148


In [ ]:
tourney_matchups = pd.read_pickle("../ncaa_data/tourney_matchups.p")

In [ ]:
probs = model.predict_proba(tourney_matchups[feature_columns].fillna(0))

In [ ]:
csv_file = open("submission.csv", "w+")
team_ids = list(tourney_matchups["Team1"])
team_ids2 = list(tourney_matchups["Team2"])
seasons = list(tourney_matchups["Season"])
csv_file.write("ID,Pred\n")
for i in range(len(tourney_matchups["Season"])):
    csv_string = str(seasons[i]) + "_" + str(team_ids[i]) + "_" + str(int(team_ids2[i]))  + "," + str(probs[i][1]) + "\n"
    csv_file.write(csv_string)

In [ ]:
csv_file.close()

# Final Model and Predictions

In [ ]:
#For a split with only the last tournament as test data
train_df = populated_df
x_train = train_df[feature_columns]
y = populated_df["Team1Score"] - populated_df["Team2Score"]
y_train = train_df["Team1Score"] - train_df["Team2Score"]
y_train[y_train < 0] = 0
y_train[y_train > 0] = 1
y[y < 0] =0
y[y> 0 ] =1

model = GradientBoostingClassifier(n_estimators=100, max_depth=3)
model.fit(x_train.fillna(0), y_train)

In [ ]:
tourney_matchups = pd.read_pickle("../ncaa_data/final_matchups.p")

In [ ]:
probs = model.predict_proba(tourney_matchups[feature_columns].fillna(0))

In [ ]:
csv_file = open("submission3-18.csv", "w+")
team_ids = list(tourney_matchups["Team1"])
team_ids2 = list(tourney_matchups["Team2"])
seasons = list(tourney_matchups["Season"])
csv_file.write("ID,Pred\n")
for i in range(len(tourney_matchups["Season"])):
    csv_string = str(seasons[i]) + "_" + str(team_ids[i]) + "_" + str(int(team_ids2[i]))  + "," + str(probs[i][1]) + "\n"
    csv_file.write(csv_string)

In [ ]:
csv_file.close()

In [ ]:
predictions = pd.concat([pd.Series([prob[0] for prob in probs]), pd.Series(team_ids), pd.Series(team_ids2)], axis=1, ignore_index=True)

In [64]:
predictions = pd.concat([pd.Series([prob[0] for prob in probs]), pd.Series(team_ids), pd.Series(team_ids2)], axis=1, ignore_index=True)
predictions.columns = ['Predicted', 'team1' , 'team2']
team_data = pd.read_csv("../ncaa_data/womens-machine-learning-competition-2019/WTeams.csv")
predictions = predictions.join(team_data.set_index('TeamID'), on='team2', how='left').join(team_data.set_index('TeamID'), lsuffix="team1", on='team1', how='left')

In [65]:
predictions[['TeamNameteam1', 'TeamName', 'Predicted']].to_csv('brackets.csv')